In [ ]:
from main_library import *

################################
## data load / adjustements*
extractDate = date_load()

### pour l'instant ne fonctionne pas
## demander à Eric de relancer la machine sur sandbox
# get_call_info()

# get_call_info_europa()

proj = projects_load()
proj_id_signed = proj.project_id.unique()

prop = proposals_load()
proj = proj_add_cols(prop, proj)

stage_p =  ['REJECTED' ,'NO_MONEY' ,'MAIN', 'RESERVE', 'INELIGIBLE', 'WITHDRAWN', 'INADMISSIBLE', None]
prop1 = proposals_status(prop, proj_id_signed, stage_p)  
# np.save("data_files/applicants_columns.npy", prop_cols)

###########################################
# proposals fix
# projects missing from proposals
call_to_integrate, call_miss = proposals_id_missing(prop1, proj, extractDate)

# project data missing in proposals if call already in proposals then add this
proj1 = proj_id_miss_fixed(prop1, proj, call_to_integrate)
call_miss = list(set(call_miss)-set(call_to_integrate))
proj = proj.loc[~proj.callId.isin(call_miss)]

# merge proj + prop
print('### MERGED PROPOSALS/PROJECTS')
if len(proj1)==0:
    prop2=pd.concat([proj,prop1], ignore_index= True)
else:
    prop2 = pd.concat([prop1, proj1, proj], ignore_index = True)

prop2 = prop2.loc[~((prop2.status_code=='REJECTED')&(prop2.stage=='successful'))]
print(f"- result - merged all: {len(prop2)},\n{prop2[['stage','status_code']].value_counts()}")

merged = copy.deepcopy(prop2)
merged = dates_year(merged)
merged = strings_v(merged)
merged = url_to_clean(merged)
merged.mask(merged=='', inplace=True)
merged = empty_str_to_none(merged)      
merged.rename(columns={
    'freekw':'free_keywords',
    'callDeadlineDate':'call_deadline', 
    'callId':'call_id', 
    'submissionDate':'submission_date',
    'startDate':'start_date',
    'endDate':'end_date', 
    'ecSignatureDate':'signature_date'}, inplace=True)

if any(merged.loc[merged.stage=='successful', 'project_id'].value_counts()[merged.loc[merged.stage=='successful', 'project_id'].value_counts()> 1]):
    print(merged.loc[merged.stage=='successful', 'project_id'].value_counts()[merged.loc[merged.stage=='successful', 'project_id'].value_counts()> 1])

merged = merged_panels(merged)
merged = merged_topics(merged)
merged = merged_actions(merged)

# calls list
calls = call(PATH_SOURCE+FRAMEWORK+'/')

print("\n### CALLS+MERGED")
if len(merged.loc[merged.call_id.isnull()])>0:
        print(f"1 - ATTENTION : manque des call_id: {merged.loc[merged.call_id.isnull(), 'project_id']}")
else:
    call_id = merged[['call_id', 'call_deadline']].drop_duplicates()
    print(f"2 - CALL_ID de merged -> nb call+deadline: {len(call_id)}, nb call unique: {call_id.call_id.nunique()} ")

calls = calls_to_check(calls, call_id)

projects = projects_complete_cleaned(merged, extractDate)

#############################################################
##### PARTICIPATIONS
part = participants_load(proj)
# conserve uniquement les projets présents dans proposals et applicants
part = part.loc[part.project_id.isin(projects.project_id.unique())]
print(f"- size part hors proj manquant: {len(part)}")
part = part_role_type(part)
part = erc_role(part, projects)

#### APPLICANTS
app = applicants_load(prop)
# conserve uniquement les projets présents dans proposals et applicants
app1 = app.loc[app.project_id.isin(projects.project_id.unique())] 
print(f"- size app1 hors proj exclus: {len(app1)}")

app_missing_pid = projects.loc[(projects.stage=='evaluated')&(~projects.project_id.isin(app1.project_id.unique())), 'project_id'].unique()
tmp = part[part.project_id.isin(app_missing_pid)]
app1 = part_miss_app(tmp, app1)

#redressement accelerator
acc_folio = pd.read_csv(f"{PATH_SOURCE}{FRAMEWORK}/eic_fund_portfolio.csv", sep=';', dtype={'PROPOSAL_NBR':str})
print(f"size acc_folio: {len(acc_folio)}")
acc = (app1.loc[(app1.project_id.isin(acc_folio.PROPOSAL_NBR.unique()))&(app1.role=='Coordinator'),['project_id', 'role']]
       .merge(acc_folio[['PROPOSAL_NBR','GRANT_REQUESTED']], how='inner', left_on='project_id', right_on='PROPOSAL_NBR')
       .drop(columns='PROPOSAL_NBR'))
print(f"size acc: {len(acc)}")
app1 = app1.merge(acc, how='left', on=['project_id', 'role'])
app1.loc[app1.requestedGrant.isnull(), 'requestedGrant'] = app1.GRANT_REQUESTED
app1.drop(columns=['GRANT_REQUESTED'], inplace=True)

app1 = app_role_type(app1)
app1 = erc_role(app1, projects)

del app

####
# verification Etat des participations
part = check_multiP_by_proj(part)
app1 = check_multiA_by_proj(app1)


### STEP2
lien = merged_partApp(app1, part)
lien = nuts_lien(app1, part, lien)
lien.to_pickle(f"{PATH_CLEAN}lien.pkl")

# ENTITIES
entities = entities_load(lien)
entities_single = entities_single_create(entities, lien)
entities_info = entities_info_create(entities_single, lien)

list_codeCountry = entities_info.countryCode.unique()
countries = country_load(FRAMEWORK, list_codeCountry)

### step3

# ##################################
# # nouvelle actualisation ; à executer UNE FOIS
# ref_source = ref_source_load('ref')
# result, check_id_liste, identification = first_update(ref_source, entities_info, countries)

# # vérifier dans excel les nouveaux ID PATH_WORK/_check_id_result.xlsx
# IDchecking_results(result, check_id_liste, identification)

# id_verified = ID_resultChecked()
# new_ref_source(id_verified, ref_source, extractDate, part, app1, entities_single, countries)

# ########################################################################################################

# chargement du nouveau ref_source
ref_source = ref_source_load('ref')
ref, genPic_to_new = ref_source_2d_select(ref_source, 'HE')
entities_tmp = entities_tmp_create(entities_info, countries, ref)
print(f"size entities_tmp: {len(entities_tmp)}")
entities_tmp = entities_for_merge(entities_tmp)

### Executer uniquement si besoin
# lid_source, unknow_list = ID_entities_list(ref_source)
# ror = ror_getRefInfo(lid_source)
# siren_siret = get_siret_siege(lid_source)
# paysage_id = ID_to_IDpaysage(lid_source, siren_siret)
# paysage, paysage_mires = paysage_getRefInfo(lid_source, siren_siret, paysage_old=None)
# paysage_category = IDpaysage_category(paysage)
# sirene = get_sirene(lid_source, sirene_old=None)

#############################################################################################################


### merge entities_tmp + referentiel
# ROR
### si besoin de charger ror pickle
ror = pd.read_pickle(f"{PATH_REF}ror_df.pkl")
entities_tmp = merge_ror(entities_tmp, ror)

# PAYSAGE
### si besoin de charger paysage pickle
paysage = pd.read_pickle(f"{PATH_REF}paysage_df.pkl")
paysage_category = pd.read_pickle(f"{PATH_SOURCE}paysage_category.pkl")
# paysage_category = IDpaysage_category(paysage)
cat_filter = category_paysage(paysage_category)
entities_tmp = merge_paysage(entities_tmp, paysage, cat_filter)
sirene = pd.read_pickle(f"{PATH_REF}sirene_df.pkl")
entities_tmp = merge_sirene(entities_tmp, sirene)

### LAST DATE of EXTRACTED DATA
[{'extraction_date': '2024-10-09', 'framework': 'HORIZON'}]

### LOADING PROJECTS data
- no new columns
- result -> dowloaded projects:17382, retained projects:17382, pb:0
- liste des colonnes conservées:
Index(['project_id', 'callId', 'callDeadlineDate', 'typeOfActionCode',
       'acronym', 'status_code', 'startDate', 'endDate', 'ecSignatureDate',
       'title', 'abstract', 'url', 'topicCode', 'duration', 'total_cost',
       'eu_reqrec_grant', 'number_involved', 'framework', 'lastUpdateDate',
       'totalGrant', 'nationalContribution', 'otherContribution', 'freekw',
       'stage'],
      dtype='object')

### LOADING PROPOSALS data
- no new columns
1- empty cols -> Attention ! vérifier les variables manquantes->[]
- result -> dowloaded proposals:92864, retained proposals:92864, pb:0

### ADD COLS TO PROJECTS FROM PROPOSALS
- liste des variables PROJ en plus: ['nationalContribution', 'startDate', 'otherContribution', 'status_code', 'endDate', 'ecSign

c:\Users\zfriant\Documents\GitHub\pcri\.venv\pcri-env\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
c:\Users\zfriant\Documents\GitHub\pcri\.venv\pcri-env\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


- code panel existant pour d'autres programmes que erc/msca: données exportées

- size merged after add panels: 102295
### TOPICS
1 - topics -> 2448
2 - divisions -> 3172
erc : destination_code à null après traitement
['HORIZON-ERC-2021-VICECHAIRS-IBA' 'ERC-2025-NCPS-IBA'
 'HORIZON-ERC-2021-arXiv-IBA'
 'HORIZON-CL4-2024-SSA-SST-SD-IBA-to-be-deleted'
 'HORIZON-CL4-2024-SSA-SST-AE-IBA-to-be-deleted'
 'HORIZON-ERC-2023-VICECHAIRS-IBA' 'HORIZON-ERC-2022-VICECHAIRS-IBA'
 'HORIZON-CL4-2024-SSA-SST-SP-IBA-to-be-deleted']
MSCA : destination_code à null après traitement
['HORIZON-MSCA-2021-INCO-01-01' 'HORIZON-MSCA-2021-FRC-IBA'
 'HORIZON-MSCA-2022-ALUMNI-IBA' 'HORIZON-MSCA-2024-NCP-01-01'
 'HORIZON-MSCA-2023-FTP-01-01' 'HORIZON-MSCA-2022-Ukraine-ART195-IBA'
 'HORIZON-MSCA-2021-RR-01-01' 'HORIZON-MSCA-2024-RR-01-01'
 'HORIZON-MSCA-2023-BEL-IBA' 'HORIZON-MSCA-2024-UKRAINE-IBA'
 'HORIZON-MSCA-2024-INCO-01-01' 'HORIZON-MSCA-SNLS-2021-IBA'
 'HORIZON-MSCA-2021-NCP-01-01' 'HORIZON-MSCA-2024-ALUMNI-IB

c:\Users\zfriant\Documents\GitHub\pcri\step1_mainData\topics.py:171: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'SESAR' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  top.loc[mask, 'destination_code'] = i


size merged after add topics: 102295

### ACTIONS
1- call sans action:
                                      call_id
24693  HORIZON-HLTH-2022-DISEASE-06-two-stage

- size merged after add actions: 102295


### CALLS

### CALLS+MERGED
2 - CALL_ID de merged -> nb call+deadline: 530, nb call unique: 492 

### CALLS to CHECK
- CALLS de base calls with merge call_id ->
nb call+deadline: 537, nb call unique: 492 
2- si calls pas dans call_id :
                             call_id call_deadline
231  HORIZON-EIC-2023-BOOSTER-IBA-01    2023-07-05
248  HORIZON-EIC-2024-ACCELERATOR-02    2024-10-03
251  HORIZON-EIC-2024-BOOSTER-IBA-01    2024-09-04
252  HORIZON-EIC-2024-BOOSTER-IBA-01    2024-11-14
467       HORIZON-MSCA-2021-SNLS-IBA    2021-07-27
505    HORIZON-WIDERA-2022-ACCESS-07    2022-04-20
522    HORIZON-WIDERA-2023-ACCESS-06    2024-09-26
3- calls with multi deadline
                                     call_id call_deadline   call_budget
17                              ERC-2023-POC    

c:\Users\zfriant\Documents\GitHub\pcri\api_requests\paysage.py:536: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  paysage_category['category_end'] = pd.to_datetime(paysage_category['category_end'], format='mixed', errors='ignore')


In [16]:
entities_tmp = entities_tmp_create(entities_info, countries, ref)
print(f"size entities_tmp: {len(entities_tmp)}")
entities_tmp = entities_for_merge(entities_tmp)

### Executer uniquement si besoin
# lid_source, unknow_list = ID_entities_list(ref_source)
# ror = ror_getRefInfo(lid_source)
# siren_siret = get_siret_siege(lid_source)
# paysage_id = ID_to_IDpaysage(lid_source, siren_siret)
# paysage, paysage_mires = paysage_getRefInfo(lid_source, siren_siret, paysage_old=None)
# paysage_category = IDpaysage_category(paysage)
# sirene = get_sirene(lid_source, sirene_old=None)

#############################################################################################################


### merge entities_tmp + referentiel
# ROR
### si besoin de charger ror pickle
ror = pd.read_pickle(f"{PATH_REF}ror_df.pkl")
entities_tmp = merge_ror(entities_tmp, ror)

# PAYSAGE
### si besoin de charger paysage pickle
paysage = pd.read_pickle(f"{PATH_REF}paysage_df.pkl")
paysage_category = pd.read_pickle(f"{PATH_SOURCE}paysage_category.pkl")
# paysage_category = IDpaysage_category(paysage)
cat_filter = category_paysage(paysage_category)
entities_tmp = merge_paysage(entities_tmp, paysage, cat_filter)
sirene = pd.read_pickle(f"{PATH_REF}sirene_df.pkl")
entities_tmp = merge_sirene(entities_tmp, sirene)

entities_tmp.loc[(~entities_tmp.id.isnull())&(entities_tmp.entities_id.isnull()), 'entities_id'] = entities_tmp.id

if any(entities_tmp.siren.str.contains(';', na=False)):
    print("ATTENTION faire code pour traiter deux siren différents -> ce qui serait bizarre qu'il y ait 2 siren")

### create ENTITIES TMP pour ref
- size entities_info before:77991, size entities_info+ref -> tmp:77991, generalPic unique:77919
- entities_info en + -> (tmp2): 0
- End size entities_tmp 77991
size entities_tmp: 77991
1 - After add ref to entities: 77991

Index(['generalPic', 'legalName', 'businessName', 'id', 'id_secondaire',
       'ZONAGE', 'country_code_mapping', 'countryCode_parent'],
      dtype='object')
2 - size entities si multi id -> entities_size_to_keep = 77995
### merge ROR
size entities_tmp after add ror_info: 77995
### CATEGORY paysage
### merge PAYSAGE
- doublons PIC      generalPic                                          legalName  \
98     876941596                       WEIR MINERALS EUROPE LIMITED   
99     876941596                       WEIR MINERALS EUROPE LIMITED   
152    877256167                                  EURACTIV MEDIA BV   
153    877256167                             EURACTIV MEDIA NETWORK   
299    877625834  NEPTUNE LINES SHIPPING AND MANAGING EN

In [18]:
import pandas as pd
def IDpic(entities_tmp):
    print("\n### traitement id_pic avec tiret")
    # IDENT with '-' : traitement des identifiants avec '-' pour regrouper multi-pic non identifiés 
    if not entities_tmp.loc[entities_tmp.id.str.contains('-', na=False)].empty:
        pic_dash = (entities_tmp.loc[entities_tmp.id.str.contains('-', na=False), ['generalPic', 'entities_id']]
        .drop_duplicates()
        .drop(columns='generalPic')
        .assign(pic_d = entities_tmp.entities_id.str.split('-').str[0]))
        print(f"- size entities pic_dash: {len(pic_dash)}")
        dash = (pic_dash.merge(entities_tmp, how='inner', left_on='pic_d', right_on='generalPic', suffixes=['_x',''])
                .drop(columns=['entities_id_x', 'pic_d'])).drop_duplicates()
        
        entities_tmp = entities_tmp.loc[~entities_tmp.entities_id.isin(dash.entities_id.unique())]
        entities_tmp = pd.concat([entities_tmp, dash], ignore_index=True)
        print(f"- size entities_tmp: {len(entities_tmp)}")

    # IDENT pic : corriger appliquer les lignes ci-dessous uniuqument sur entities_id est null ou commence par pic
    entities_tmp.loc[entities_tmp.entities_id.isnull(), 'entities_id'] = "pic"+entities_tmp.generalPic.map(str)
    print(f"- size entities_tmp: {len(entities_tmp)}")
    return entities_tmp
entities_tmp = IDpic(entities_tmp)


### traitement id_pic avec tiret
- size entities_tmp: 77995


In [19]:
entities_tmp = entities_tmp.merge(get_source_ID(entities_tmp, 'entities_id'), how='left', on='entities_id')

### sourcer les identifiants pour getInformations


In [37]:
def groupe_treatment(df, output):
    import pandas as pd, numpy as np, openpyxl, warnings, copy
    warnings.simplefilter("ignore")
    PATH_REF = "C:/Users/zfriant/Documents/OneDrive/PCRI/eCorda_datas/datas_reference/"

    liste = pd.read_excel(f"{PATH_REF}_groupes_liste.xlsx", dtype=object, keep_default_na=False, sheet_name = "liste")
    ge = openpyxl.load_workbook(f"{PATH_REF}{df}.xlsm").sheetnames[1:]
#     ge = liste_pcri
    
    gr = pd.DataFrame()
    verif = pd.DataFrame()

    for i in ge:
        x = pd.read_excel(f"{PATH_REF}{df}.xlsm", sheet_name=i, dtype=str)
        
        if len(x)>0:
            x.dropna(axis = 0, how = 'all', inplace = True)
            
            if 'Identifiant unité légale' in x.columns:
                x = x.rename(columns={'Identifiant unité légale':'siren'})
            elif 'Unité légale' in x.columns:
                x = x.rename(columns={'Unité légale':'siren'})
                
            if 'Unité légale étrangère ?' in x.columns:  
                x = x.loc[x['Unité légale étrangère ?']=="Non"]  
            else:
                pass
 
            if 'Taux détention' in x.columns:
                x = x.assign(detention = x['Taux détention'].str.replace(',', '.').astype(float))
            elif 'Taux integration' in x.columns:
                x = x.assign(detention = x['Taux integration'].str.replace(',', '.').astype(float))
                
            x = x.loc[~(x['detention'] < 50.)]
                      
            print(i, end=",")
            verif = pd.concat([verif, x], ignore_index=True)
        

            x['GROUPE'] = i
            x = x.merge(liste, how='inner', on="GROUPE")
            gr = pd.concat([gr, x], ignore_index=True)

    print(f"\n1 - Nb de groupes dans gr: {gr.ordre.nunique()}\nGroupes non traités (n'existent plus): {set(ge)-set(gr.GROUPE.unique())}")
    
    # verif_na <- gr[apply(gr, 2, function(x) any(is.na(x)))]
    if gr[gr.siren.isnull()].empty:
        pass
    else:
        print(f"2 - Attention des siren sont null\n{gr.loc[gr.siren.isnull(), ['Raison sociale', 'groupe_acronym', 'ordre']]}")
        gr=gr.loc[~gr.siren.isnull()]

    # # contrôle de la longueur des siren ; ajout de 0 devant si < 9
    for i in gr.columns:
        if gr[i].dtype == 'str':
            gr[i] = gr[i].map(str.strip)
        else:
            pass
        
    
    if any(9-gr.siren.str.len())>0:
        gr['siren'] = gr['siren'].str.rjust(9, fillchar='0')
    else:
        print(f"3 - autre pb avec le siren {gr[gr.siren.str.len()!=9][['siren', 'GROUPE', 'long']]}")


    groupe = copy.deepcopy(gr)[['siren', 'Etat', 'Date de fin', 'GROUPE', 'ordre', 'ex_groupe', 'groupe_name', 'groupe_acronym', 'groupe_sector']]
    print(f"4 - size groupe {len(groupe)}")

    groupe['n'] = groupe.groupby('siren', dropna=False)['siren'].transform('count')

    groupe = groupe.loc[~((groupe.n>1) & ((groupe.Etat.isin(["Cessée", "Inactive économique", "Inactive statistique"])) | ~(groupe['Date de fin'].isnull())))]
    groupe['n'] = groupe.groupby('siren', dropna=False)['siren'].transform('count')

    if any(groupe.n>1):
        print(f"vérifier dans groupe les doublons n>1\n{groupe[groupe.n>1]}")
    else:
        print(f"ok -> {len(groupe)}")


    groupe['groupe_id'] = "gent"+groupe.ordre.map(str)
    groupe = groupe[['siren', 'groupe_name', 'groupe_acronym', 'groupe_id', 'groupe_sector']]    
    groupe.siren = groupe.siren.astype(str)
    
    file_name = f"{PATH_REF}{output}.pkl"
    with open(file_name, 'wb') as file:
        pd.to_pickle(groupe, file)
    
    return groupe
groupe = groupe_treatment('groupe_prov', 'groupe')
# wb=openpyxl.load_workbook(f"{PATH_REF}groupe_prov.xlsm").sheetnames[1:]
### si besoin de charger groupe 
groupe = pd.read_pickle(f"{PATH_REF}groupe.pkl")
print(f"taille de entities_tmp avant groupe:{len(entities_tmp)}")
# entities_tmp = merge_groupe(entities_tmp, groupe)

accor,acri,ad industries,aeroports de paris,agence reuters,air france,air liquide,airbus,airbusdefense,airbushelico,akira,akka technologies,alcen,alfa,algosource,alpha,alstom,altair,alten,amplitude laser,anteric,aperam,aqualande,arcelormittal,areva,ariane,arjowiggins,arkema,artal technologies,artelia,asfalp,assystem,atos,avions,awt,axa,bayer,bconstr,bimmo,bio rad,bnp,bonduelle,brgm sa,btelecom,ca,canon,capgemini,caraholding,cargill,carrefour,centum electronics,clarivate analytics,cma,cmi,cnim,colas,collecte localisation satellite,constellium,continental,coriolis composites,cs,danone,dassault,dcns,decathlon,doris,draka,ds smith,duvarry,edfRTE,eiffage,elvia,engie,equip aero,eramet,ericsson,ernst,esi,essilor,esterline,eurofins,faiveley,faurecia,fibre,fives,flex,foritech,geenergie,gemedical,giat,gobain,h51,hewlett,hitachi,hitec premium,horiba,hp,idemia,ifv,ineos,intel,ixcore,xlabo meteo essai,lactalis,lafarge,latecoere,legrand,lesieur,limagrain,loiretech,lord,lp2c,lvmh,lynred,mbda,merial,m

In [15]:
# entities_tmp[entities_tmp.generalPic=='888071958']
len(entities_tmp)

18733